In [1]:
%matplotlib inline
#
from LRGSG_package.shared import *
from LRGSG_package.config.plotlib import imshow_colorbar_caxdivider
from LRGSG_package.config.utils import find_largest_cluster_circle2D
from lrgsglib.core import move_to_rootf, Lattice2D, flip_to_positive_majority_adapted
from matplotlib.colors import ListedColormap
#
move_to_rootf()
plt.style.use("ipynb/nb_plotsheet.mplstyle")
#
subDatPth = os.path.join('plot', 'paper', 'suppinf', '')
datPth = os.path.join('data', subDatPth)
subPthStr = ['defects_plots']
datPthDict = {s: os.path.join(datPth, s, '') for s in subPthStr}
for path in datPthDict.values(): os.makedirs(path, exist_ok=True)

Current working directory: /home/opisthofulax/PhD/projects/complex-networks/lrgsglib


## Eigenstate: squared, Single, Z err, X err

In [2]:
side = 96
geo = 'squared'
cmap = ListedColormap([plt.cm.viridis(0), plt.cm.viridis(0.99)])

In [3]:
for cell in ['rand', 'randZERR', 'randXERR']:
    thezip = zip([0.05, 0.103, 0.25], ['a', 'b', 'c'])
    if cell == 'randXERR':
        thezip = zip([0.103, 0.25, 0.5], ['a', 'b', 'c'])
    for pflip, ll in thezip:
        l = Lattice2D(side, geo,
            pflip=pflip,
            pbc=False,
            sgpath=subDatPth,
            initNwDict=True,
            with_positions=True
        )
        #
        l.flip_sel_edges(l.nwDict[cell]['G'])
        l.compute_k_eigvV()
        fig, ax = plt.subplots(ncols=2)
        #
        ax[1].set_aspect('equal')
        #
        ticks = [0, side-1]
        ticksl = [r'$1$', fr'${side}$']
        #
        arr = flip_to_positive_majority_adapted(l.bin_eigV().reshape(l.syshape))
        im = ax[0].imshow(np.rot90(arr), cmap=cmap, origin='upper', interpolation='none')
        #
        ax[0].set_xticks(ticks)
        ax[0].set_yticks(ticks)
        ax[0].set_xticklabels(ticksl)
        ax[0].set_yticklabels(ticksl)
        #
        ax[0].text(0.05, 0.9, fr'\textbf{{({ll})}}', transform=ax[0].transAxes, fontsize=50, 
                    fontweight='bold', color='black', ha='left', va='center', 
                    bbox=dict(facecolor='white', alpha=0.8, edgecolor='none', pad=5))
        #
        divdr, caxm, cbar = imshow_colorbar_caxdivider(im, ax[0])
        cbar.set_ticks([])
        #
        if cell == 'rand' or cell == 'randZERR':
            nx.draw(l.G, pos = l.get_node_pos(), width=1, node_color='black',
                edge_color=l.get_edge_color(pec='none'), ax=ax[1], node_size=1)
            if cell == 'rand':
                circles = [tuple([i[0][0], i[0][1]+.5]) if (i[0][1] == i[1][1] - 1) else tuple([i[0][0]+.5, i[0][1]]) for i in l.rEdgeFlip['G']]
                radius = l.r_c
                largest_cluster = find_largest_cluster_circle2D(circles, radius)
                node_1 = set(largest_cluster)
                node_all = set(circles)
                node_oher = node_all - node_1
                for i in circles:
                    test = (i[0], i[1])
                    circle = plt.Circle(test, radius, color='k' if test in node_oher else 'r', alpha=.25)
                    ax[1].add_patch(circle)
            elif cell == 'randZERR':
                circles = np.array(l.nwDict.rNodeFlip['G'])  # Ensure this is an array of tuples
                radius = l.r_c
                largest_cluster = find_largest_cluster_circle2D(circles, radius)
                node_1 = set(largest_cluster)
                node_all = set(l.nwDict.rNodeFlip['G'])
                node_oher = node_all - node_1
                #
                for i in list(node_all):
                    circle_center = (i[0]-.5, i[1]-.5)
                    circle = plt.Circle(circle_center, radius, color='k' if i in node_oher else 'r', alpha=.25)
                    ax[1].add_patch(circle)
        elif cell == 'randXERR':
            l.load_eigV_on_graph()
            node_colors =l.get_node_attr('eigV0')
            nx.draw(l.G, pos = l.get_node_pos(), width=2, edgecolors='black',
                edge_color=l.get_edge_color(pec='none'), ax=ax[1], node_size=0, node_color=flip_to_positive_majority_adapted(np.sign(l.get_node_attr('eigV0'))), cmap=cmap)
        fig.tight_layout()
        fig.savefig(f"{datPthDict['defects_plots']}{geo}_{side}_{pflip}_{cell}.png", 
            bbox_inches='tight', transparent=True, dpi=300)
        plt.close('all')

## Other

In [ ]:
side = 96
pflip = 0.103
geometry = 'squared'
cell = 'rand'

In [ ]:
configurations = [
    (side, True),
    (32, False)
]
#
for i, (side1, pbc) in enumerate(configurations):
    l_tmp = Lattice2D(
        side1=side1,
        geo=geometry,
        pflip=pflip,
        pbc=pbc,
        sgpath=subDatPth,
        with_positions=True,
        initNwDict=True
    )
    #
    fliplist = l_tmp.nwDict[cell]['G']
    l_tmp.flip_sel_edges(fliplist)
    l_tmp.compute_k_eigvV()
    #
    if i == 0:
        l = l_tmp
    elif i == 1:
        l_draw = l_tmp

In [ ]:
fig, ax = plt.subplots(ncols=2)
ax[0].imshow(np.sign(l.eigV[0].reshape(l.syshape)))
nx.draw(l_draw.G, pos = l_draw.get_node_pos(), 
    width=0, ax=ax[1], node_size=10)
ax[1].set_aspect('equal')
if cell == 'rand':
    for i in l_draw.rEdgeFlip['G']:
        if i[0][1] == i[1][1] - 1:
            circle_center = (i[0][0], i[0][1]+.5)
        elif i[0][0] == i[1][0] - 1:
            circle_center = (i[0][0]+.5, i[0][1])
        circle = plt.Circle(circle_center, l_draw.r_c, color='red', alpha=0.25)
        ax[1].add_patch(circle)
elif cell == 'randXERR':
    for circle_center in l_draw.nwDict.rNodeFlip['G']:
        circle = plt.Circle(circle_center, l_draw.r_c, color='red', alpha=0.25)
        ax[1].add_patch(circle)
elif cell == 'randZERR':
    for i in l_draw.nwDict.rNodeFlip['G']:
        circle_center = (i[0]-.5, i[1]-.5)
        circle = plt.Circle(circle_center, l_draw.r_c, color='red', alpha=0.25)
        ax[1].add_patch(circle)
fig.tight_layout()
fig.savefig(f"{datPthDict['defects_plots']}{geometry}_{side}_{pflip}_{cell}.pdf", 
    bbox_inches='tight', transparent=True)
# plt.close('all')

In [ ]:
rows, cols= 64, 64
p = 0.1
random_matrix = np.random.rand(rows*cols)
binary_matrix = np.where(random_matrix < p, 1, -1)


In [ ]:
rows, cols= 128, 128
pinf = []
for p in np.linspace(0.1, 0.9, 50):
    random_matrix = np.random.rand(rows, cols)
    binary_matrix = np.where(random_matrix < p, 1, -1)
    pinf.append(np.abs(np.where(binary_matrix.flatten() == 1)[0].shape[0] / (rows*cols) - np.where(binary_matrix.flatten() == -1)[0].shape[0] / (rows*cols)))

In [ ]:
plt.plot(np.linspace(0.1, 0.9, 50), pinf)

In [ ]:
plt.imshow(binary_matrix)

In [ ]:
l = Lattice2D(64)
nx.set_node_attributes(l.G, values=random.choices([-1, 1], k=l.G.number_of_nodes()))

In [ ]:
l = Lattice2D(2048, pflip=0.001, with_positions=False)
l.flip_sel_edges(l.nwDict['rand']['G'])

In [ ]:
l.compute_k_eigvV()